In [1]:
import sys
from tqdm import tqdm
import pandas as pd
import numpy as np
from torch.nn import functional as F
import torch

sys.path.insert(0, '../')

from inference import predict_submission
from model import load_model
from dataset import get_dataloader, LabelEncoder

In [2]:
ageg = pd.read_csv('../prediction/all2021-04-07 162133/submission_ageg.csv')
gender = pd.read_csv('../prediction/all2021-04-07 162133/submission_gender.csv')
mask = pd.read_csv('../prediction/all2021-04-07 162133/submission_mask.csv')

submission = pd.DataFrame(dict(ImageID=ageg['ImageID']))

In [5]:
submission['mask'] = mask['ans']
submission['gender'] = gender['ans']
submission['ageg'] = ageg['ans']
enc = LabelEncoder()

In [7]:
submission['ans'] = submission.apply(lambda x: enc.transform(tuple(x.iloc[1:]), task='main'), axis=1)

In [10]:
submission[['ImageID', 'ans']].to_csv('submission_ens_enc.csv', index=False)

In [7]:
main.to_csv('submission_ens.csv', index=False)

In [2]:
TASKS = ['mask', 'gender', 'ageg']
CLASSES = {'mask':3, 'gender':2, 'ageg':3}
WEIGHTS = {'mask':6, 'gender':3, 'ageg':1}

param_dict = dict()
param_dict['ageg'] = 'VanillaEfficientNet_task(ageg)ep(02)f1(0.8562)loss(0.0008)lr(0.001)trans(random)optim(adamp)crit(labelsmoothingLoss)seed(42).pth'
param_dict['mask'] = 'VanillaEfficientNet_task(mask)ep(09)f1(0.9845)bs(64)loss(0.0007)lr(0.005)trans(base)optim(adam)crit(labelsmoothingLoss)seed(42).pth'
param_dict['gender'] = 'VanillaEfficientNet_task(gender)ep(08)f1(0.9736)bs(32)loss(0.0006)lr(0.001)trans(random)optim(adamp)crit(labelsmoothingLoss)seed(42).pth'

predictions = dict()

for t in TASKS:
    load_state_dict = os.path.join('../saved_models', t, param_dict[t])
    model_type = param_dict[t].split('_')[0]
    transform_type = param_dict[t].split('trans(')[-1].split(')')[0]
    model = load_model(model_type=model_type, task=t, load_state_dict=load_state_dict)
    model.cpu()
    model.eval()

    dataloader = get_dataloader(
        task=t,
        phase="eval",
        data_root='../input/data/eval/images',
        transform_type=transform_type,
        # batch_size=1024,
        batch_size=4,
        shuffle=False,
        drop_last=False
    )

    with torch.no_grad():
        id_list = []
        pred_list = []

        for img_ids, imgs in tqdm(dataloader, desc="Inference"):
            imgs = imgs.cpu()
            output = model(imgs)
            preds = F.softmax(output, dim=1)
            pred_list.append(preds)
            id_list.extend(img_ids)
    
    prediction = pd.DataFrame(np.vstack(pred_list))
    prediction['ImageID'] = id_list

    prediction.to_csv(
        os.path.join(f"prob_{t}.csv"), index=False
    )

Loaded pretrained weights for efficientnet-b3
Inference:   0%|          | 0/3150 [00:06<?, ?it/s]


In [9]:
preds_np = preds.numpy()
preds_np.shape

(4, 3)

In [8]:
np.vstack([preds_np, preds_np]).shape

(8, 3)